In [1]:
#import scrapy
#from scrapy.crawler import CrawlerProcess


#class ESSpider(scrapy.Spider):
#    name = "ESS"
    
#    letters = ['A','B','C','D','E','F','G','H','I','J','K','L','M', 'N','O','P','Q','R','S','T','U','V','W','X','Y','Z']
#    start_urls = []
#    for letter in letters:
#        start_urls.append('https://bishop-accountability.org/priestdb/PriestDBbylastName-{}.html'.format(letter))

#    def parse(self, response):
#       with open('mainpage.html', 'wb') as f:
#            f.write(response.body)
            
#            for article in response.xpath('//tr'):
#            
#                yield {
#                    'other': article.xpath('td/text()').extract(),
#                    'name': article.xpath('td/b').extract()
#                }

#process = CrawlerProcess({
#    'FEED_FORMAT': 'json',         # Store data in JSON format.
#    'FEED_URI': 'raw_data.json',  # Name our storage file.
#    'LOG_ENABLED': False,           # Turn off logging for now.
#    'DOWNLOAD_DELAY' : 3
#})


#process.crawl(ESSpider)
#process.start()


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import plotly.figure_factory as ff

In [3]:
raw_data = pd.read_json('/Users/jdobrow/Desktop/Data Science/CatholicAbuse/raw_data.json')
#https://bishop-accountability.org/priestdb/PriestDBbylastName-A.html

drop_list = []
for i in range(len(raw_data.name)):
    if raw_data.name[i] == []:
        drop_list.append(i)
raw_data.drop(drop_list, 0, inplace=True)
raw_data.reset_index(inplace=True, drop=True)
raw_data.drop(6410, 0, inplace=True)
raw_data.reset_index(inplace=True, drop=True)
drop_list = []
for i in range(len(raw_data.other)):
    if raw_data.other[i] == []:
        drop_list.append(i)
raw_data.drop(drop_list, 0, inplace=True)
raw_data.reset_index(inplace=True, drop=True)

In [4]:
clean_data = raw_data.copy()

In [5]:
first_names = []
last_names = []
for row in range(len(raw_data.name)):
    last_names.append(raw_data.name[row][0][3:][:-4])
    first_names.append(raw_data.name[row][1][3:][:-4])
clean_data['FirstName'] = first_names
clean_data['LastName'] = last_names
clean_data.drop('name', 1, inplace=True)

In [6]:
bad_rows = []
for row in range(len(clean_data.other)):
    if len(clean_data.other[row][1]) > 1:
        bad_rows.append(row)

In [7]:
date_list = []
for row in range(len(clean_data.other)):
    if row in bad_rows:
        date_list.append(None)
    else:
        date_list.append(clean_data.other[row][0])
clean_data['YearOrdained'] = date_list

In [8]:
type_list = []
for row in range(len(clean_data.other)):
    if row in bad_rows:
        type_list.append(clean_data.other[row][0])
    else:
        type_list.append(clean_data.other[row][1])
clean_data['Type'] = type_list
# - T = Type. P=Priest, B=Brother, S=Seminarian, D=Deacon, N=Nun

In [9]:
status_list = []
for row in range(len(clean_data.other)):
    if row in bad_rows:
        status_list.append(clean_data.other[row][1])
    else:
        status_list.append(clean_data.other[row][2])
clean_data['Status'] = status_list

In [10]:
employed_list = []
for row in range(len(clean_data.other)):
    if row in bad_rows:
        employed_list.append(clean_data.other[row][2])
    else:
        employed_list.append(clean_data.other[row][3])
clean_data['EmployedBy'] = employed_list
# - T = Type. P=Priest, B=Brother, S=Seminarian, D=Deacon, N=Nun

In [11]:
notes_list = []
for row in range(len(clean_data.other)):
    if row in bad_rows:
        notes_list.append(clean_data.other[row][3])
    else:
        notes_list.append(clean_data.other[row][4])
clean_data['Notes'] = notes_list
# - T = Type. P=Priest, B=Brother, S=Seminarian, D=Deacon, N=Nun

In [12]:
diocese_list = []
for row in range(len(clean_data.other)):
    if row in bad_rows:
        diocese_list.append(clean_data.other[row][4])
    else:
        try:
            diocese_list.append(clean_data.other[row][5])
        except:
            diocese_list.append(None)
clean_data['Diocese'] = diocese_list
# - T = Type. P=Priest, B=Brother, S=Seminarian, D=Deacon, N=Nun

In [13]:
clean_data.drop('other', 1, inplace=True)

In [14]:
state_list = []
city_list = []
for row in range(len(clean_data.Diocese)):
    try:
        state_list.append(clean_data.Diocese[row].split()[-1])
        city = ''
        for i in clean_data.Diocese[row].split()[:-1]:
            city += i
            city += ' '
        city = city[:-2]
        city_list.append(city)
    except:
        state_list.append(None)
        city_list.append(None)
        
clean_data['DioceseState'] = state_list
clean_data['DioceseCity'] = city_list
clean_data.drop('Diocese', 1, inplace=True)

In [15]:
years = []

for row in clean_data.YearOrdained:
    try:
        if len(row) == 4:
            years.append(int(row))
        else:
            years.append(-1)
    except:
        years.append(-1)
clean_data['YearOrdained'] = years

In [16]:
clean_data

,FirstName,LastName,YearOrdained,Type,Status,EmployedBy,Notes,DioceseState,DioceseCity
0,Rubin,Abaya,-1,P,Accused,Diocesan,One of seven priests named as defendants in ...,CA,Los Angeles
1,Andrew,Abdon,1945,B,Settled,Bros Christian School,Aka John Christianson. In separate 1995 lawsui...,NM,Santa Fe
2,Leonard A.,Abercrombie,1946,P,Accused,Diocesan,"Letter 7/93 to Pope JP II, Stafford, and Mahon...",CO,Denver
3,Lionel Augustine,Abeywickrema,1951,P,Accused,Diocesan,From Sri Lanka. Accused by three sisters of se...,NM,Santa Fe
4,Phillip,Abinate,-1,P,Accused,Diocesan,Named publicly as accused on the diocese's lis...,CA,Monterey
5,John L.,Abrams,1951,P,Accused,Diocesan,'Sick leave '1989 until 1997 when 'retired.' S...,NY,Brooklyn
6,Joseph A.,Abruzzese,1990,P,Convicted,Diocesan,Placed on leave 12/1/93. Pled no contest in 19...,RI,Providence
7,Anthony,Acciarito,1978,P,Accused,Diocesan,Named publicly as accused by the diocese on it...,NY,Brooklyn
8,Jeffrey N.,Acebo,1986,P,Sued,Diocesan,"As a newly ordained priest, Acebo confessed to...",CA,Oakland
9,Francis,Ackerson,1946,P,Accused,Diocesan,Monsignor. Named in the 3/1/16 PA Grand Jury R...,PA,Altoona-Johnstown


In [17]:
map_data = pd.DataFrame()
map_data['States'] = clean_data.DioceseState.unique()
amount_list = []
for state in map_data.States:
    amount_list.append(len(clean_data[clean_data.DioceseState == state]))
map_data['Number'] = amount_list

In [18]:
state_pop = pd.read_csv('/Users/jdobrow/Desktop/Data Science/CatholicAbuse/state-population.txt')
state_pop = state_pop[(state_pop.year == 2012) & (state_pop.ages == 'total')]
state_pop.reset_index(inplace=True, drop=True)
state_pop.drop([51, 52], inplace=True)
state_pop.reset_index(inplace=True, drop=True)

In [23]:
# https://www.pewforum.org/religious-landscape-study
state_codes = ['RI', 'MA', 'NJ', 'NM', 'CT', 'NY', 'CA', 'IL', 'LA', 'NH', 'ND', 'NV', 'PA', 'NE', 'TX', 'DE', 'MN', 'SD', 'VT', 'AZ', 'FL', 'ME', 'DC', 'IN', 'IA', 'KS', 'MI', 'OH', 'MT', 'WA', 'AK', 'CO', 'MO', 'MD', 'OR', 'VA', 'ID', 'KY', 'SC', 'GA', 'NC', 'AR', 'OK', 'AL', 'TN', 'WV', 'UT', 'MS', 'HI', 'WI', 'WY']
state_percents = [42, 34, 34, 34, 33, 31, 28, 28, 26, 26, 26, 25, 24, 23, 23, 22, 22, 22, 21 ,21 ,21, 20, 20, 18, 18, 18, 18, 18, 17, 17, 16, 16, 16, 15, 12, 12, 10, 10, 10, 9, 9, 8, 8, 7, 6, 6, 5, 4, 20, 25, 14]
states = pd.DataFrame()
states['StateCodes'] = state_codes
states['PercentCatholic'] = np.array(state_percents) * 0.01

In [24]:
pop_count = []
for i in range(len(map_data.States)):
    try:
        pop_count.append(state_pop.population[state_pop['state/region'] == map_data.States[i]].iloc[0])
    except:
        pop_count.append(None)
map_data['Population'] = pop_count

percent_count = []
for i in range(len(map_data.States)):
    try:
        percent_count.append(states.PercentCatholic[states['StateCodes'] == map_data.States[i]].iloc[0])
    except:
        percent_count.append(None)
map_data['PercentCatholic'] = percent_count

In [25]:
map_data['NumberCatholics'] = (map_data.Population * map_data.PercentCatholic)
map_data['AccusedToNumberCatholics'] = 1000000 * map_data.Number/(map_data.Population * map_data.PercentCatholic)
map_data

,States,Number,Population,PercentCatholic,NumberCatholics,AccusedToNumberCatholics
0,CA,657,37999878.0,0.28,10639965.84,61.748319
1,NM,109,2083540.0,0.34,708403.60,153.867089
2,CO,34,5189458.0,0.16,830313.28,40.948400
3,NY,721,19576125.0,0.31,6068598.75,118.808316
4,RI,55,1050304.0,0.42,441127.68,124.680455
5,PA,556,12764475.0,0.24,3063474.00,181.493298
6,TX,279,26060796.0,0.23,5993983.08,46.546678
7,FL,102,19320749.0,0.21,4057357.29,25.139516
8,MA,401,6645303.0,0.34,2259403.02,177.480510
9,WI,169,5724554.0,0.25,1431138.50,118.087802


In [26]:
import plotly.graph_objects as go

fig = go.Figure(data=go.Choropleth(
    locations=map_data.States,
    z = map_data.AccusedToNumberCatholics.astype(float),
    locationmode = 'USA-states',
    colorscale = 'Reds',
    colorbar_title = 'Offenders per Million Catholics',
))

fig.update_layout(
    title_text = 'Accused Priests by Catholic Population ',
    geo_scope='usa',
)

fig.show()